<a href="https://colab.research.google.com/github/ErickPower/CPSC501-Assignment4/blob/master/PART2/notMNIST-NotOptimized.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git init
!git remote add origin https://github.com/ErickPower/CPSC501-Assignment4.git
!git pull origin master

Initialized empty Git repository in /content/.git/
remote: Enumerating objects: 63, done.
remote: Counting objects: 100% (63/63), done.
remote: Compressing objects: 100% (53/53), done.
remote: Total 63 (delta 29), reused 6 (delta 2), pack-reused 0
Unpacking objects: 100% (63/63), done.
From https://github.com/ErickPower/CPSC501-Assignment4
 * branch            master     -> FETCH_HEAD
 * [new branch]      master     -> origin/master


In [2]:
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  print("Tensorflow already loaded.")
  pass

TensorFlow 2.x selected.


Upload notMNIST.npz

In [0]:
#from google.colab import files
#uploaded = files.upload()

In [11]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

print("--Get data--")
with np.load("notMNIST.npz", allow_pickle=True) as f:
    x_train, y_train = f['x_train'], f['y_train']
    x_test, y_test = f['x_test'], f['y_test']

print("--Process data--")
print(len(y_train))
x_train, x_test = x_train / 255.0, x_test / 255.0

#x_train, x_test = x_train.reshape(x_train.shape[0], 28, 28, 1),  x_test.reshape(x_test.shape[0], 28, 28, 1)
 
print("--Make model--")
model = tf.keras.models.Sequential([
  
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  #tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(600, activation='relu'),
  tf.keras.layers.Dense(400, activation='relu'),
  tf.keras.layers.Dense(200, activation='relu'),
  tf.keras.layers.Dense(10, activation='softmax')
])

model.summary()

optim = tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.9)

model.compile(optimizer=optim, loss='sparse_categorical_crossentropy', metrics=['accuracy'])


print("--Fit model--")
model.fit(x_train, y_train, epochs=30, verbose=2)

print("--Evaluate model--")
model_loss, model_acc = model.evaluate(x_test,  y_test, verbose=2) 
print(f"Model Loss:    {model_loss:.2f}")
print(f"Model Accuray: {model_acc*100:.1f}%")

--Get data--
--Process data--
60000
--Make model--
Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_5 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_20 (Dense)             (None, 600)               471000    
_________________________________________________________________
dense_21 (Dense)             (None, 400)               240400    
_________________________________________________________________
dense_22 (Dense)             (None, 200)               80200     
_________________________________________________________________
dense_23 (Dense)             (None, 10)                2010      
Total params: 793,610
Trainable params: 793,610
Non-trainable params: 0
_________________________________________________________________
--Fit model--
Train on 60000 samples
Epoch 1/30
60000/60000 - 1

In [0]:
#Save Model to notMNIST.h5
model.save("notMNIST.h5")

In [17]:
#push model to github


[master 45c5a68] Adding notMNIST.h5 to github.
Counting objects: 5, done.
Delta compression using up to 2 threads.
Compressing objects: 100% (5/5), done.
Writing objects: 100% (5/5), 5.69 MiB | 6.14 MiB/s, done.
Total 5 (delta 2), reused 0 (delta 0)
remote: Resolving deltas: 100% (2/2), completed with 1 local object.
To https://github.com/ErickPower/CPSC501-Assignment4.git
   077c44b..45c5a68  master -> master


In [0]:
def plot_image(i, predictions_array, true_label, img):
  predictions_array, true_label, img = predictions_array, true_label[i], img[i]
  plt.grid(False)
  plt.xticks([])
  plt.yticks([])

  plt.imshow(img, cmap=plt.cm.binary)

  predicted_label = np.argmax(predictions_array)
  if predicted_label == true_label:
    color = 'blue'
  else:
    color = 'red'

  plt.xlabel("{} {:2.0f}% ({})".format(class_names[predicted_label],
                                100*np.max(predictions_array),
                                class_names[true_label]),
                                color=color)

def plot_value_array(i, predictions_array, true_label):
  predictions_array, true_label = predictions_array, true_label[i]
  plt.grid(False)
  plt.xticks(range(10))
  plt.yticks([])
  thisplot = plt.bar(range(10), predictions_array, color="#777777")
  plt.ylim([0, 1])
  predicted_label = np.argmax(predictions_array)

  thisplot[predicted_label].set_color('red')
  thisplot[true_label].set_color('blue')

In [0]:
class_names = ['A','B','C','D','E','F','G','H','I','J']

In [0]:
predictions = model.predict(x_test)
num_rows = 6
num_cols = 3
num_images = num_rows*num_cols
plt.figure(figsize=(2*2*num_cols, 2*num_rows))
for i in range(num_images):
  plt.subplot(num_rows, 2*num_cols, 2*i+1)
  plot_image(i+num_images, predictions[i+num_images], y_test, x_test)
  plt.subplot(num_rows, 2*num_cols, 2*i+2)
  plot_value_array(i+num_images, predictions[i+num_images], y_test)
plt.tight_layout()
plt.show()

In [0]:
#from google.colab import files
#files.download('notMNIST.h5')

My original model consists of 5 layers; 1 flatten layer (to reshape data from 28x28 to 1x784), and 4 Dense layers. The dense layers are of decreasing size outputs (600, 400, 200, 10), and the first 3 use the Rectified Linear Unit activation function.

The fourth dense layer uses softmax activation function, as this is best for outputting a prediction between 0 and 1 for each possible result. Since the output size of this layer is 10, it is producing a prediction for the input (image) to be classified into each possible category (A-J, 10 categories).

I used 30 epochs as this got to an acceptable level of accuracy (98% train, 94% test)

After drawing the letter H (in imageH.png), I tested the model on this image. It incorrectly labelled it as D.

#Changes:

